# Evolver Loop 9 Analysis

## Key Question: Why is the saspav baseline at 70.66 while target is 68.92?

The gap of 1.74 points (2.5%) cannot be closed by local optimization. We need to understand:
1. What techniques could close this gap?
2. Are there better baselines we haven't found?
3. What do top teams do differently?

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob

# Check all available CSV files in snapshots
all_csvs = glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f'Found {len(all_csvs)} CSV files in snapshots')

Found 730 CSV files in snapshots


In [2]:
# Tree geometry for scoring
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def compute_score_for_n(df, n):
    prefix = f'{n:03d}_'
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    
    all_points = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        angle_rad = np.radians(deg)
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        for tx, ty in zip(TX, TY):
            px = tx * cos_a - ty * sin_a + x
            py = tx * sin_a + ty * cos_a + y
            all_points.append([px, py])
    
    all_points = np.array(all_points)
    side = max(all_points.max(axis=0) - all_points.min(axis=0))
    return side**2 / n

def compute_total_score(df):
    return sum(compute_score_for_n(df, n) for n in range(1, 201))

print('Scoring functions defined')

Scoring functions defined


In [3]:
# Compare with saspav baseline
df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
baseline_score = compute_total_score(df_baseline)
print(f'Saspav baseline score: {baseline_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap: {baseline_score - 68.919154:.6f} ({(baseline_score - 68.919154)/68.919154*100:.2f}%)')

Saspav baseline score: 70.659959
Target score: 68.919154
Gap: 1.740805 (2.53%)


In [4]:
# Analyze per-N scores to find where improvements are possible
per_n_scores = {}
for n in range(1, 201):
    per_n_scores[n] = compute_score_for_n(df_baseline, n)

# Theoretical minimum for tree packing
# Tree width = 0.7, height = 1.0 (from -0.2 to 0.8)
# For N=1, minimum side = 1.0 (height), score = 1.0/1 = 1.0
# Actual N=1 score is much lower due to rotation

print('\nPer-N score analysis:')
print(f'N=1: {per_n_scores[1]:.6f}')
print(f'N=2: {per_n_scores[2]:.6f}')
print(f'N=10: {per_n_scores[10]:.6f}')
print(f'N=50: {per_n_scores[50]:.6f}')
print(f'N=100: {per_n_scores[100]:.6f}')
print(f'N=200: {per_n_scores[200]:.6f}')


Per-N score analysis:
N=1: 0.661250
N=2: 0.450779
N=10: 0.376630
N=50: 0.360753
N=100: 0.345531
N=200: 0.337564


In [5]:
# Calculate contribution to total score by N range
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(per_n_scores[n] for n in range(start, end+1))
    pct = range_score / baseline_score * 100
    print(f'N={start}-{end}: {range_score:.4f} ({pct:.1f}% of total)')

N=1-10: 4.3291 (6.1% of total)
N=11-50: 14.7126 (20.8% of total)
N=51-100: 17.6323 (25.0% of total)
N=101-150: 17.1408 (24.3% of total)
N=151-200: 16.8452 (23.8% of total)


In [6]:
# The key insight: to close the 1.74 gap, we need to improve by ~2.5%
# If improvements are uniform, each N needs to improve by 2.5%
# But small N have more room for improvement

print('\nRequired improvement per N range to close gap:')
target = 68.919154
gap = baseline_score - target

for start, end in ranges:
    range_score = sum(per_n_scores[n] for n in range(start, end+1))
    range_gap = gap * (range_score / baseline_score)  # Proportional share
    pct_improvement = range_gap / range_score * 100
    print(f'N={start}-{end}: need {range_gap:.4f} improvement ({pct_improvement:.2f}%)')


Required improvement per N range to close gap:
N=1-10: need 0.1067 improvement (2.46%)
N=11-50: need 0.3625 improvement (2.46%)
N=51-100: need 0.4344 improvement (2.46%)
N=101-150: need 0.4223 improvement (2.46%)
N=151-200: need 0.4150 improvement (2.46%)


In [7]:
# Summary
print('='*60)
print('ANALYSIS SUMMARY')
print('='*60)
print(f'Current best: {baseline_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {baseline_score - 68.919154:.6f} (2.5%)')
print()
print('KEY FINDINGS:')
print('1. Local optimization (SA, bbox3, Eazy) provides ~0 improvement')
print('2. The saspav baseline is at an extremely strong local optimum')
print('3. Gap of 1.74 points requires fundamentally different approaches')
print()
print('POTENTIAL APPROACHES:')
print('1. Tree removal technique (chistyakov) - extract smaller N from larger N')
print('2. Different construction algorithms (not starting from saspav)')
print('3. Much longer optimization runs (hours, not minutes)')
print('4. Ensemble from multiple independent constructions')
print('5. Find better pre-optimized solutions from other sources')

ANALYSIS SUMMARY
Current best: 70.659959
Target: 68.919154
Gap: 1.740805 (2.5%)

KEY FINDINGS:
1. Local optimization (SA, bbox3, Eazy) provides ~0 improvement
2. The saspav baseline is at an extremely strong local optimum
3. Gap of 1.74 points requires fundamentally different approaches

POTENTIAL APPROACHES:
1. Tree removal technique (chistyakov) - extract smaller N from larger N
2. Different construction algorithms (not starting from saspav)
3. Much longer optimization runs (hours, not minutes)
4. Ensemble from multiple independent constructions
5. Find better pre-optimized solutions from other sources
